In [1]:
import os
import sys
import pandas as pd
import dtale

import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime, timedelta

pd.set_option('display.max_columns', None)

In [3]:
# file_path = r'data\health\email\email_data.parquet'
# data = pd.read_parquet(file_path)
# print(data.shape)
# print(data.columns)
# data.head()

In [5]:
file_path = r'data\health\email_label.parquet'
# Reading the data for validation
data = pd.read_parquet(file_path)
print(data.shape)

(10296, 13)


In [8]:
d = dtale.show(data)
d.open_browser()

In [6]:
# Renaming the columns
data.rename(columns={
    "label": "label",
    "score": "score"
}
          , inplace=True)

In [7]:
data.columns

Index(['id', 'from_email', 'to_email', 'cc_email', 'subject', 'body',
       'date_ist', 'time_ist', 'projectid', 'clean_email', 'sentiment_NLTK',
       'label', 'score'],
      dtype='object')

In [6]:
# converting the date column to date type
data['date_ist'] = pd.to_datetime(data['date_ist'])

In [7]:
data.head(2)

,id,subject,date_ist,time_ist,projectid,label,score
0,6d30273a-6d16-4c9a-b207-31261df388ba,RE: 23-044 Drafting,2024-05-16,05:13:05,76461,POSITIVE,0.5574
1,6b7a51f9-201f-43fa-adc2-f23203ee00d5,RE: 23-043 Drafting,2024-05-18,03:34:35,76461,NEUTRAL,0.0000


In [9]:
print(data.shape)
print('Unique Projects: ', data['projectid'].nunique())
data['projectid'].value_counts().head()

(10362, 7)
Unique Projects:  717


projectid
64591    960
63901    233
78785    136
77043    126
77552    120
Name: count, dtype: int64

In [13]:
# Calculate weights for individual emails based on their sentiment scores
def calculate_weight(row):
    if row['label'] == 'NEGATIVE':
        if -1.0 <= row['score'] <= -0.8:
            return 10
        elif -0.8 < row['score'] <= -0.5:
            return 5
        else:
            return abs(row['score'])
    else:
        return row['score']
    

data['weighted_score'] = data.apply(calculate_weight, axis=1)

data.head(5)

,id,subject,date_ist,time_ist,projectid,label,score,weighted_score
0,6d30273a-6d16-4c9a-b207-31261df388ba,RE: 23-044 Drafting,2024-05-16,05:13:05,76461,POSITIVE,0.5574,0.5574
1,6b7a51f9-201f-43fa-adc2-f23203ee00d5,RE: 23-043 Drafting,2024-05-18,03:34:35,76461,NEUTRAL,0.0000,0.0000
2,4c74d3e9-230c-4f6b-9e36-3a9986d162d6,RE: Exhaust Louver Size - McMaster,2024-05-13,21:12:48,76187,POSITIVE,0.3182,0.3182
3,af6c2697-de52-4bca-9454-bd9ec04286e8,Re: [EXTERNAL] Re: ABMS Hangar,2024-04-24,21:55:04,77901,NEUTRAL,0.0000,0.0000
4,33feb1a9-3b8f-4d95-82ee-d5accabc96da,RE: Tristar Vault 3D Model,2024-04-18,05:43:17,75205,POSITIVE,0.4576,0.4576


In [14]:
data[data['label'] =='NEGATIVE'].head(5)

,id,subject,date_ist,time_ist,projectid,label,score,weighted_score
16,48a2348b-7c83-4467-9c35-0b87addc35cd,WD Manor Trimble,2024-05-16,00:30:03,78616,NEGATIVE,-0.3939,0.3939
50,fce7d129-fd71-496a-88f5-054b0a1a7158,RE: 722075 Harper Bentley,2024-04-27,03:43:07,75705,NEGATIVE,-0.3740,0.3740
51,9b331a68-ff56-40dc-9c70-54e3d1663e4d,RE: New Project 724014,2024-05-10,05:25:19,75705,NEGATIVE,-0.2411,0.2411
98,d295633b-74ef-4d70-8c96-a04e7aeb6b4c,RE: R3B: LEX files for review.,2024-04-23,22:30:32,78523,NEGATIVE,-0.2960,0.2960
130,ddfebb75-b071-4833-a559-9a9044999228,SRI Deliverables week of 05/28/2024,2024-05-28,20:53:24,74935,NEGATIVE,-0.5052,5.0000


In [41]:
# Function to aggregate data, add counts, and tag project labels
def aggregate_and_tag_data(df):
    agg_data = df.groupby('projectid').agg(
        total_count=('score', 'count'),
        positive_count=('label', lambda x: (x == 'POSITIVE').sum()),
        neutral_count=('label', lambda x: (x == 'NEUTRAL').sum()),
        negative_count=('label', lambda x: (x == 'NEGATIVE').sum()),
        negative_weight=('weighted_score', lambda x: sum(x[df['label'] == 'NEGATIVE'])),
        neutral_weight=('weighted_score', lambda x: sum(x[df['label'] == 'NEUTRAL'])),
        positive_weight=('weighted_score', lambda x: sum(x[df['label'] == 'POSITIVE']))
    ).reset_index()

    # Calculate total weighted sentiment for each project
    agg_data['total_weighted_sentiment'] = agg_data['negative_weight'] + agg_data['neutral_weight'] + agg_data['positive_weight']

    # Tagging function with check for total_weighted_sentiment being zero
    def tag_project_label(row):
        if row['total_weighted_sentiment'] == 0:
            return 'NEUTRAL'  # or some default label
        if row['negative_weight'] / row['total_weighted_sentiment'] > 0.3:  # Adjust threshold as needed
            return 'NEGATIVE'
        elif row['neutral_weight'] > row['positive_weight']:
            return 'NEUTRAL'
        else:
            return 'POSITIVE'

    agg_data['project_label'] = agg_data.apply(tag_project_label, axis=1)

    return agg_data

In [42]:
# Aggregating and tagging the data for overall project label
agg_data_overall = aggregate_and_tag_data(data)

agg_data_overall.head(5)

,projectid,total_count,positive_count,neutral_count,negative_count,negative_weight,neutral_weight,positive_weight,total_weighted_sentiment,project_label
0,10111,1,1,0,0,0.0000,0.0000,0.5574,0.5574,POSITIVE
1,10121,48,36,10,2,0.6372,0.0258,20.0818,20.7448,POSITIVE
2,10134,19,16,0,3,1.1797,0.0000,12.3984,13.5781,POSITIVE
3,10178,1,1,0,0,0.0000,0.0000,0.5813,0.5813,POSITIVE
4,10282,8,8,0,0,0.0000,0.0000,4.9301,4.9301,POSITIVE


In [43]:
# Filter the data for recent emails (last 10 days)
recent_threshold_date = pd.to_datetime('now') - timedelta(days=10)
df_recent = data[data['date_ist'] >= recent_threshold_date]

# Aggregating and tagging the data for recent project label
agg_data_recent = aggregate_and_tag_data(df_recent)
agg_data_recent = agg_data_recent.rename(columns={'project_label': 'recent_project_label'})

In [44]:
agg_data_recent.head()

,projectid,total_count,positive_count,neutral_count,negative_count,negative_weight,neutral_weight,positive_weight,total_weighted_sentiment,recent_project_label
0,10111,1,1,0,0,0.0000,0.0000,0.5574,0.5574,POSITIVE
1,10121,4,3,1,0,0.0000,0.0258,1.7492,1.7750,POSITIVE
2,10134,2,2,0,0,0.0000,0.0000,1.7036,1.7036,POSITIVE
3,10282,1,1,0,0,0.0000,0.0000,0.4051,0.4051,POSITIVE
4,10314,4,0,3,1,0.2263,0.0000,0.0000,0.2263,NEGATIVE


In [45]:
# Merge the overall and recent labels into a single DataFrame
combined_agg_data = agg_data_overall[['projectid', 'project_label']].merge(
    agg_data_recent[['projectid', 'recent_project_label']], 
    on='projectid', 
    how='left'
)

combined_agg_data.head()

,projectid,project_label,recent_project_label
0,10111,POSITIVE,POSITIVE
1,10121,POSITIVE,POSITIVE
2,10134,POSITIVE,POSITIVE
3,10178,POSITIVE,NaN
4,10282,POSITIVE,POSITIVE


In [46]:
combined_agg_data.shape

(717, 3)

In [37]:
# print('Overall:' ,agg_data_overall.shape)
# print('Recent:', agg_data_recent.shape)
# print('Merged:', combined_agg_data.shape)

In [38]:
# print('overll:',agg_data_overall['project_label'].value_counts())
# print('Recent:',agg_data_recent['recent_project_label'].value_counts())

In [47]:
print('Merged:',combined_agg_data['project_label'].value_counts())
print('Merged_recent:',combined_agg_data['recent_project_label'].value_counts())

Merged: project_label
POSITIVE    586
NEGATIVE    112
NEUTRAL      19
Name: count, dtype: int64
Merged_recent: recent_project_label
POSITIVE    182
NEGATIVE     27
NEUTRAL      15
Name: count, dtype: int64
